In [2]:
import math
import networkx as nx
import osmnx as ox
import osmnx.distance as od
import matplotlib.pyplot as plt
import random
import time

In [3]:
def autoSetup(centre, radius, generatorCount):
    start = time.time()
    print("AUTOMATIC SETUP PROCEDURE")
    G = ox.graph_from_point(centre, dist=radius, network_type="walk")
    Gp = ox.project_graph(G)
    print("Total nodes: " + str(len(list(Gp))))
    generators = []
    for i in range(generatorCount):
        generators.append(list(Gp)[random.randint(0,len(Gp))])
    end = time.time()
    timing(start, end)
    return generators, Gp

def manualSetup(centre, radius, generatorList):
    start = time.time()
    print("MANUAL SETUP PROCEDURE")
    G = ox.graph_from_point(centre, dist=radius, network_type="walk")
    xarray = []
    yarray = []
    for i in generatorList:
        xarray.append(i[1])
        yarray.append(i[0])
    generators = od.nearest_nodes(G, xarray, yarray)
    Gp = ox.project_graph(G)
    print("Total nodes: " + str(len(list(Gp))))
    end = time.time()
    timing(start, end)
    return generators, Gp

def network(generators, Gp):
    start = time.time()
    missed = 0
    print()
    print("NETWORK ASSIGNMENT PROCEDURE")
    grlist = [ [] for _ in range(len(generators)) ]
    for i in list(Gp):
        sdists = 50000 # Very large distance
        for j in generators:
            try:    
                rtl = nx.shortest_path_length(Gp, i, j, weight="length")
            except:
                rtl = 50000 # Very large distance
                missed += 1
            if rtl < sdists:
                sdists = rtl
                sj = generators.index(j)
        print(str(round(100*list(Gp).index(i)/(len(list(Gp))-1),2)) + "% complete", end="\r")
        for k in grlist:
            if sj == grlist.index(k):
                k.append(i)
    print(str(len(list(Gp)) * len(generators) - missed) + " routes calculated.")
    if missed > 0:
        print("WARNING: " + str(missed) + " potential route(s) unreachable.")
    end = time.time()
    timing(start, end)
    return grlist

def euclid(generators, Gp):
    start = time.time()
    missed = 0
    print()
    print("EUCLIDEAN ASSIGNMENT PROCEDURE")
    grlist = [ [] for _ in range(len(generators)) ]
    for i in list(Gp):
        sdists = 50000 # Very large distance
        i_x = Gp.nodes[i]['x']
        i_y = Gp.nodes[i]['y']
        for j in generators:
            j_x = Gp.nodes[j]['x']
            j_y = Gp.nodes[j]['y']
            try:    
                rtl = od.euclidean(i_y, i_x, j_y, j_x)
            except:
                rtl = 50000 # Very large distance
                missed += 1
            if rtl < sdists:
                sdists = rtl
                sj = generators.index(j)
        print(str(round(100*list(Gp).index(i)/(len(list(Gp))-1),2)) + "% complete", end="\r")
        for k in grlist:
            if sj == grlist.index(k):
                k.append(i)
    print(str(len(list(Gp)) * len(generators) - missed) + " distances calculated.")
    end = time.time()
    timing(start, end)
    return grlist

def taxicab(generators, Gp, griddir):
    start = time.time()
    missed = 0
    print()
    print("MANHATTAN ASSIGNMENT PROCEDURE")
    grlist = [ [] for _ in range(len(generators)) ]
    xarray = []
    yarray = []
    for i in griddir:
        xarray.append(i[1])
        yarray.append(i[0])
    gdnodes = od.nearest_nodes(Gp, xarray, yarray)
    dy0 = abs(yarray[1] - yarray[0])
    dx0 = abs(xarray[1] - xarray[0])
    rho = math.atan(dx0/dy0)
    for i in list(Gp):
        sdists = 50000 # Very large distance
        i_x = Gp.nodes[i]['x']
        i_y = Gp.nodes[i]['y']
        for j in generators:
            if i == j:
                sdists = 0
                sj = generators.index(j)
                break
            j_x = Gp.nodes[j]['x']
            j_y = Gp.nodes[j]['y']
            try:    
                dx = abs(i_x - j_x)
                dy = abs(i_y - j_y)
                hyp = math.sqrt(dx ** 2 + dy ** 2)
                phi = math.atan(dx/dy) - rho
                rtl = hyp * (math.cos(phi) + math.sin(phi))
            except:
                rtl = 50000 # Very large distance
                missed += 1
            if rtl < sdists:
                sdists = rtl
                sj = generators.index(j)
        print(str(round(100*list(Gp).index(i)/(len(list(Gp))-1),2)) + "% complete", end="\r")
        for k in grlist:
            if sj == grlist.index(k):
                k.append(i)
    print(str(len(list(Gp)) * len(generators) - missed) + " distances calculated.")
    print("Grid bearing: " + str(round((180/math.pi)*rho,2)) + "°")
    end = time.time()
    timing(start, end)
    return grlist

def comparison(test, base):
    start = time.time()
    print()
    print("COMPARISON PROCEDURE")
    match = 0
    total = 0
    mismatch = []
    for i in test:
        for j in i:
            mmcheck = 0
            total += 1
            for k in base[test.index(i)]:
                if j == k:
                    match += 1
                    mmcheck += 1
                    break
            if mmcheck == 0:
                mismatch.append(j)
    acc = round(100*(match/total),2)
    end = time.time()
    print(str(total) + " nodes analysed. Total accuracy of test compared to base: " + str(acc) + "%.")
    print(str(len(mismatch)) + " misclassifications in total.")
    timing(start, end)
    return mismatch, acc

def graphIt(graph, base, inColours, title):
    start = time.time()
    print()
    print("GRAPHING PROCEDURE")
    print(title + ".png:")
    poly = []
    outColours = []
    done = 0
    for i in base:
        for j in i:
            poly.append(ox.shortest_path(graph, j, j))
            outColours.append(inColours[base.index(i)])
    fig, ax = ox.plot_graph_routes(graph, poly, route_colors=outColours, route_linewidths=6, node_size = 0)
    fig.savefig(title + ".png")
    end = time.time()
    timing(start, end)
        
def timing(s, e):
    flag = False
    timediff = round(e - s, 2)
    if timediff > 60:
        mins = round(math.floor(timediff/60), 2)
        secs = round(timediff - 60 * mins, 0)
        if str(secs)[1] == ".":
            psecs = "0" + str(secs)[0]
            flag = True
        if flag != True:
            print("Time elapsed: " + str(mins) + ":" + str(secs) + ".")
        else:
            print("Time elapsed: " + str(mins) + ":" + psecs + ".")
    else:
        print("Time elapsed: " + str(timediff) + "s.")

In [7]:
# Use Ctrl+/ to comment/un-comment code.
# This cell is the London example

centre = (51.513374,-0.089216) # Bank Junction, City of London
generatorCount = 5
radius = 800
colours = ["#00dd00", "#ff0000", "#ff8800", "#ffdd00", "#008800"]

generatorList, outGraph = autoSetup(centre, radius, generatorCount)

outEuclid = euclid(generatorList, outGraph)
outNetwork = network(generatorList, outGraph)

misclassEN, accEN = comparison(outEuclid, outNetwork)

combinedEN = [] # Create a combined list for graphIt
combinedEN.append(generatorList)
combinedEN.append(misclassEN)

# graphIt(outGraph, combinedEN, colours, "london_gm_euclidean")
# graphIt(outGraph, outEuclid, colours, "london_euclidean")
# graphIt(outGraph, outNetwork, colours, "london_network")

In [ ]:
# This cell is the Brooklyn example

centre = (40.678174,-73.944114) # Atlantic Ave/Brooklyn Ave, Brooklyn, NY
gridDirection = [ [40.676601,-73.944390], [40.677370,-73.944283] ]
generatorCount = 5
radius = 800
colours = ["#00dd00", "#ff0000", "#ff8800", "#ffdd00", "#008800"]

generatorList, outGraph = autoSetup(centre, radius, generatorCount)

outEuclid = euclid(generatorList, outGraph)
outTaxicab = taxicab(generatorList, outGraph, gridDirection)
outNetwork = network(generatorList, outGraph)

misclassEN, accEN = comparison(outEuclid, outNetwork)
misclassTN, accTN = comparison(outTaxicab, outNetwork)

combinedEN = [] # Create a combined list for graphIt
combinedEN.append(generatorList)
combinedEN.append(misclassEN)
combinedTN = [] # Create a combined list for graphIt
combinedTN.append(generatorList)
combinedTN.append(misclassTN)

# graphIt(outGraph, combinedEN, colours, "brooklyn_gm_euclidean")
# graphIt(outGraph, combinedTN, colours, "brooklyn_gm_taxicab")
# graphIt(outGraph, outEuclid, colours, "brooklyn_euclidean")
# graphIt(outGraph, outTaxicab, colours, "brooklyn_taxicab")
# graphIt(outGraph, outNetwork, colours, "brooklyn_network")

In [ ]:
# This cell is the Northwich example

centre = (53.261970,-2.512917) # Witton St/Leicester St, Northwich, Cheshire
radius = 800
colours = ["#00dd00", "#ff0000", "#ff8800", "#ffdd00", "#008800"]
coordsList = [ [53.254691,-2.523480], [53.262286,-2.520048], [53.261544,-2.498498], [53.252174,-2.512387], [53.262290,-2.507748] ]
            #   Castle Primary School,  Winnington Rec Centre,  Northwich Station,    Viaduct at London Rd.,  Witton/Venables Sts.

generatorList, outGraph = manualSetup(centre, radius, coordsList)

outEuclid = euclid(generatorList, outGraph)
outNetwork = network(generatorList, outGraph)

misclassEN, accEN = comparison(outEuclid, outNetwork)

combinedEN = [] # Create a combined list for graphIt
combinedEN.append(generatorList)
combinedEN.append(misclassEN)

# graphIt(outGraph, combinedEN, colours, "northwich_gm_euclidean")
# graphIt(outGraph, outEuclid, colours, "northwich_euclidean")
# graphIt(outGraph, outNetwork, colours, "northwich_network")